In [1]:
import pandas as pd
import numpy as np
from random import gauss, uniform

In [11]:
def get_makespan(curr_plan, num_resources, workflow_inaccur, positive=False, dynamic_res=False):
    '''
    Calculate makespan
    '''
    under = False
    reactive_resource_usage = [0] * num_resources
    resource_usage = [0] * num_resources
    expected = [0] * num_resources
    tmp_idx = [0] * num_resources
    error = [0] * num_resources
    for placement in curr_plan:
        workflow = placement[0]
        resource = placement[1]
        resource_id = resource['id']
        expected_finish = placement[3]
        expected_start = placement[2]
        if dynamic_res:
            perf = gauss(resource['performance'], resource['performance'] * 0.0644)
        else:
            pref = resource['performance']

        if positive:
            inaccur = uniform(0, workflow_inaccur)
        else:
            inaccur = uniform(-workflow_inaccur, workflow_inaccur)

        actual_exec_time = (workflow['num_oper'] * (1 + inaccur)) / perf
        
        expected_exec_time = (expected_finish - expected_start)
        
        if actual_exec_time != expected_exec_time:
            error[resource_id - 1] += abs(actual_exec_time - expected_exec_time)
        
        actual_resource_usage[resource_id - 1] = max(actual_resource_usage[resource_id - 1] + actual_exec_time,
                                                     expected_finish)
        expected[resource_id - 1] = expected_finish
        reactive_resource_usage[resource_id - 1] += actual_exec_time
           
        tmp_idx[resource_id - 1] += 1

    return actual_resource_usage, reactive_resource_usage, expected, error

In [3]:
test_case = pd.read_csv('../Data/heft/StHeteroCampaigns_4DynHeteroResourcesHEFT.csv')
test = test_case.loc[0]
size = test['size']
planner = test['planner']
plan = eval(test['plan'])
makespan, reactive, expected, error = get_makespan(plan, 4, 0, dynamic_res=True)
print(makespan)
print(expected)
print(error)
print(max(makespan) - max(expected))

[0, 0, 7139.959321262717, 9980.548391911583]
[0, 0, 7139.959321262717, 9717.15964013397]
[0, 0, 517.7923369419887, 545.8780137138401]
263.3887517776129


In [4]:
plan

[({'description': None, 'id': 3, 'num_oper': 83632.36187551905},
  {'id': 4, 'performance': 23.516},
  0,
  3556.402529151176),
 ({'description': None, 'id': 4, 'num_oper': 76818.03491483296},
  {'id': 4, 'performance': 23.516},
  3556.402529151176,
  6823.0309912549765),
 ({'description': None, 'id': 2, 'num_oper': 76254.76555108582},
  {'id': 3, 'performance': 10.68},
  0,
  7139.959321262717),
 ({'description': None, 'id': 1, 'num_oper': 68058.32930703841},
  {'id': 4, 'performance': 23.516},
  6823.0309912549765,
  9717.15964013397)]

In [5]:
print(max(makespan), max(expected), max(makespan) - max(expected))

9980.548391911583 9717.15964013397 263.3887517776129


In [6]:
test = test_case.loc[100]
size = test['size']
planner = test['planner']
plan = eval(test['plan'])
makespan, reactive, expected, error = get_makespan(plan, size, 0, dynamic_res=True)
print(max(makespan), max(expected), max(makespan) - max(expected), error)

19892.58189493126 19892.58189493126 0.0 [0, 0, 1282.1753337897799, 950.1124007054032, 0, 0, 0, 0]


In [9]:
test_case = pd.read_csv('../Data/heft/DynHeteroResources_StHeteroCampaignsHEFT.csv')
test = test_case.loc[0]
size = test['size']
planner = test['planner']
plan = eval(test['plan'])
makespan, reactive, expected, error = get_makespan(plan, size, 0, dynamic_res=True)
print(makespan)
print(expected)
print(error)

[2054488.575947038, 2028138.6476615744, 2019148.0329255941, 2013577.9282695379]
[2010899.2999288826, 2011416.2091584117, 2011877.8157420638, 2012662.536056654]
[125220.68285827272, 98006.71258493, 102093.61011423424, 104316.15693381501]


In [10]:
print(max(makespan), max(expected), max(makespan) - max(expected), error)

2054488.575947038 2012662.536056654 41826.039890383836 [125220.68285827272, 98006.71258493, 102093.61011423424, 104316.15693381501]


In [ ]:
test_case = pd.read_csv('../Data/random/DynHeteroResources_StHeteroCampaignsRAND.csv')
results = pd.DataFrame(columns=['size','planner','plan','makespan', 'reactive', 'expected', 'time'])

for idx, row in test_case.iterrows():
    size = row['size']
    planner = row['planner']
    plan = eval(row['plan'])
    makespan, reactive, expected = get_makespan(plan, size, 0)
    time = row['time']
    results.loc[len(results)] = [size, planner, plan, makespan, reactive, expected, time]
results.to_csv('../Data/random/DynHeteroResources_StHeteroCampaignsRAND.csv', index=False)

test_case = pd.read_csv('../Data/random/DynHeteroResources_StHomoCampaignsRAND.csv')
results = pd.DataFrame(columns=['size','planner','plan','makespan', 'reactive','time'])

for idx, row in test_case.iterrows():
    size = row['size']
    planner = row['planner']
    plan = eval(row['plan'])
    makespan, reactive = get_makespan(plan, size, 0)
    time = row['time']
    results.loc[len(results)] = [size, planner, plan, makespan, reactive, time]
results.to_csv('../Data/random/DynHeteroResources_StHomoCampaignsRAND.csv', index=False)

test_case = pd.read_csv('../Data/random/DynHomoResources_StHeteroCampaignsRAND.csv')
results = pd.DataFrame(columns=['size','planner','plan','makespan', 'reactive','time'])

for idx, row in test_case.iterrows():
    size = row['size']
    planner = row['planner']
    plan = eval(row['plan'])
    makespan, reactive = get_makespan(plan, size, 0)
    time = row['time']
    results.loc[len(results)] = [size, planner, plan, makespan, reactive, time]
results.to_csv('../Data/random/DynHomoResources_StHeteroCampaignsRAND.csv', index=False)

test_case = pd.read_csv('../Data/random/DynHomoResources_StHomoCampaignsRAND.csv')
results = pd.DataFrame(columns=['size','planner','plan','makespan', 'reactive','time'])

for idx, row in test_case.iterrows():
    size = row['size']
    planner = row['planner']
    plan = eval(row['plan'])
    makespan, reactive = get_makespan(plan, size, 0)
    time = row['time']
    results.loc[len(results)] = [size, planner, plan, makespan, reactive, time]
results.to_csv('../Data/random/DynHomoResources_StHomoCampaignsRAND.csv', index=False)


In [ ]:
test_case = pd.read_csv('../Data/heft/StHeteroCampaigns_4DynHeteroResourcesHEFT.csv')
results = pd.DataFrame(columns=['size','planner','plan','makespan', 'reactive', 'expected', 'time'])

for idx, row in test_case.iterrows():
    size = row['size']
    planner = row['planner']
    plan = eval(row['plan'])
    makespan, reactive, expected = get_makespan(plan, size, 0)
    time = row['time']
    results.loc[len(results)] = [size, planner, plan, makespan, reactive, expected, time]
results.to_csv('../Data/heft/StHeteroCampaigns_4DynHeteroResourcesHEFT.csv', index=False)

test_case = pd.read_csv('../Data/heft/StHeteroCampaigns_4DynHomoResourcesHEFT.csv')
results = pd.DataFrame(columns=['size','planner','plan','makespan', 'reactive','time'])

for idx, row in test_case.iterrows():
    size = row['size']
    planner = row['planner']
    plan = eval(row['plan'])
    makespan, reactive = get_makespan(plan, size, 0)
    time = row['time']
    results.loc[len(results)] = [size, planner, plan, makespan, reactive, time]
results.to_csv('../Data/heft/StHeteroCampaigns_4DynHomoResourcesHEFT.csv', index=False)

test_case = pd.read_csv('../Data/heft/StHomoCampaigns_4DynFixedHomoResourcesHEFT.csv')
results = pd.DataFrame(columns=['size','planner','plan','makespan', 'reactive','time'])

for idx, row in test_case.iterrows():
    size = row['size']
    planner = row['planner']
    plan = eval(row['plan'])
    makespan, reactive = get_makespan(plan, size, 0)
    time = row['time']
    results.loc[len(results)] = [size, planner, plan, makespan, reactive, time]
results.to_csv('../Data/heft/StHomoCampaigns_4DynFixedHomoResourcesHEFT.csv', index=False)

test_case = pd.read_csv('../Data/heft/StHomoCampaigns_4DynHomoResourcesHEFT.csv')
results = pd.DataFrame(columns=['size','planner','plan','makespan', 'reactive','time'])

for idx, row in test_case.iterrows():
    size = row['size']
    planner = row['planner']
    plan = eval(row['plan'])
    makespan, reactive = get_makespan(plan, size, 0)
    time = row['time']
    results.loc[len(results)] = [size, planner, plan, makespan, reactive, time]
results.to_csv('../Data/heft/StHomoCampaigns_4DynHomoResourcesHEFT.csv', index=False)

test_case = pd.read_csv('../Data/heft/DynHeteroResources_StHeteroCampaignsHEFT.csv')
results = pd.DataFrame(columns=['size','planner','plan','makespan', 'reactive', 'expected', 'time'])

for idx, row in test_case.iterrows():
    size = row['size']
    planner = row['planner']
    plan = eval(row['plan'])
    makespan, reactive, expected = get_makespan(plan, size, 0)
    time = row['time']
    results.loc[len(results)] = [size, planner, plan, makespan, reactive, expected, time]
results.to_csv('../Data/heft/DynHeteroResources_StHeteroCampaignsHEFT.csv', index=False)

test_case = pd.read_csv('../Data/heft/DynHeteroResources_StHomoCampaignsHEFT.csv')
results = pd.DataFrame(columns=['size','planner','plan','makespan', 'reactive','time'])

for idx, row in test_case.iterrows():
    size = row['size']
    planner = row['planner']
    plan = eval(row['plan'])
    makespan, reactive = get_makespan(plan, size, 0)
    time = row['time']
    results.loc[len(results)] = [size, planner, plan, makespan, reactive, time]
results.to_csv('../Data/heft/DynHeteroResources_StHomoCampaignsHEFT.csv', index=False)

test_case = pd.read_csv('../Data/heft/DynHomoResources_StHeteroCampaignsHEFT.csv')
results = pd.DataFrame(columns=['size','planner','plan','makespan', 'reactive','time'])

for idx, row in test_case.iterrows():
    size = row['size']
    planner = row['planner']
    plan = eval(row['plan'])
    makespan, reactive = get_makespan(plan, size, 0)
    time = row['time']
    results.loc[len(results)] = [size, planner, plan, makespan, reactive, time]
results.to_csv('../Data/heft/DynHomoResources_StHeteroCampaignsHEFT.csv', index=False)

test_case = pd.read_csv('../Data/heft/DynHomoResources_StHomoCampaignsHEFT.csv')
results = pd.DataFrame(columns=['size','planner','plan','makespan', 'reactive','time'])

for idx, row in test_case.iterrows():
    size = row['size']
    planner = row['planner']
    plan = eval(row['plan'])
    makespan, reactive = get_makespan(plan, size, 0)
    time = row['time']
    results.loc[len(results)] = [size, planner, plan, makespan, reactive, time]
results.to_csv('../Data/heft/DynHomoResources_StHomoCampaignsHEFT.csv', index=False)

In [ ]:
test_case = pd.read_csv('../Data/l2ff/DynHeteroResources_StHeteroCampaignsL2FF.csv')
results = pd.DataFrame(columns=['size','planner','plan','makespan', 'reactive', 'expected', 'time'])

for idx, row in test_case.iterrows():
    size = row['size']
    planner = row['planner']
    plan = eval(row['plan'])
    makespan, reactive, expected = get_makespan(plan, size, 0)
    time = row['time']
    results.loc[len(results)] = [size, planner, plan, makespan, reactive, expected, time]
results.to_csv('../Data/l2ff/DynHeteroResources_StHeteroCampaignsL2FF.csv', index=False)

test_case = pd.read_csv('../Data/l2ff/DynHeteroResources_StHomoCampaignsL2FF.csv')
results = pd.DataFrame(columns=['size','planner','plan','makespan', 'reactive','time'])

for idx, row in test_case.iterrows():
    size = row['size']
    planner = row['planner']
    plan = eval(row['plan'])
    makespan, reactive = get_makespan(plan, size, 0)
    time = row['time']
    results.loc[len(results)] = [size, planner, plan, makespan, reactive, time]
results.to_csv('../Data/l2ff/DynHeteroResources_StHomoCampaignsL2FF.csv', index=False)

test_case = pd.read_csv('../Data/l2ff/DynHomoResources_StHeteroCampaignsL2FF.csv')
results = pd.DataFrame(columns=['size','planner','plan','makespan', 'reactive','time'])

for idx, row in test_case.iterrows():
    size = row['size']
    planner = row['planner']
    plan = eval(row['plan'])
    makespan, reactive = get_makespan(plan, size, 0)
    time = row['time']
    results.loc[len(results)] = [size, planner, plan, makespan, reactive, time]
results.to_csv('../Data/l2ff/DynHomoResources_StHeteroCampaignsL2FF.csv', index=False)

test_case = pd.read_csv('../Data/l2ff/DynHomoResources_StHomoCampaignsL2FF.csv')
results = pd.DataFrame(columns=['size','planner','plan','makespan', 'reactive','time'])

for idx, row in test_case.iterrows():
    size = row['size']
    planner = row['planner']
    plan = eval(row['plan'])
    makespan, reactive = get_makespan(plan, size, 0)
    time = row['time']
    results.loc[len(results)] = [size, planner, plan, makespan, reactive, time]
results.to_csv('../Data/l2ff/DynHomoResources_StHomoCampaignsL2FF.csv', index=False)

test_case = pd.read_csv('../Data/l2ff/StHeteroCampaigns_4DynHeteroResourcesL2FF.csv')
results = pd.DataFrame(columns=['size','planner','plan','makespan', 'reactive', 'expected', 'time'])

for idx, row in test_case.iterrows():
    size = row['size']
    planner = row['planner']
    plan = eval(row['plan'])
    makespan, reactive, expected = get_makespan(plan, size, 0)
    time = row['time']
    results.loc[len(results)] = [size, planner, plan, makespan, reactive, expected, time]
results.to_csv('../Data/l2ff/StHeteroCampaigns_4DynHeteroResourcesL2FF.csv', index=False)

test_case = pd.read_csv('../Data/l2ff/StHeteroCampaigns_4DynHomoResourcesL2FF.csv')
results = pd.DataFrame(columns=['size','planner','plan','makespan', 'reactive','time'])

for idx, row in test_case.iterrows():
    size = row['size']
    planner = row['planner']
    plan = eval(row['plan'])
    makespan, reactive = get_makespan(plan, size, 0)
    time = row['time']
    results.loc[len(results)] = [size, planner, plan, makespan, reactive, time]
results.to_csv('../Data/l2ff/StHeteroCampaigns_4DynHomoResourcesL2FF.csv', index=False)

test_case = pd.read_csv('../Data/l2ff/StHomoCampaigns_4DynFixedHomoResourcesL2FF.csv')
results = pd.DataFrame(columns=['size','planner','plan','makespan', 'reactive','time'])

for idx, row in test_case.iterrows():
    size = row['size']
    planner = row['planner']
    plan = eval(row['plan'])
    makespan, reactive = get_makespan(plan, size, 0)
    time = row['time']
    results.loc[len(results)] = [size, planner, plan, makespan, reactive, time]
results.to_csv('../Data/l2ff/StHomoCampaigns_4DynFixedHomoResourcesL2FF.csv', index=False)

test_case = pd.read_csv('../Data/l2ff/StHomoCampaigns_4DynHomoResourcesL2FF.csv')
results = pd.DataFrame(columns=['size','planner','plan','makespan', 'reactive','time'])

for idx, row in test_case.iterrows():
    size = row['size']
    planner = row['planner']
    plan = eval(row['plan'])
    makespan, reactive = get_makespan(plan, size, 0)
    time = row['time']
    results.loc[len(results)] = [size, planner, plan, makespan, reactive, time]
results.to_csv('../Data/l2ff/StHomoCampaigns_4DynHomoResourcesL2FF.csv', index=False)

In [ ]:
test_case = pd.read_csv('../Data/heft/DynHeteroResources_StHeteroCampaignsHEFT_inaccur_5perc.csv')
results = pd.DataFrame(columns=['size','planner','plan','makespan', 'reactive','time'])

for idx, row in test_case.iterrows():
    size = row['size']
    planner = row['planner']
    plan = eval(row['plan'])
    makespan, reactive = get_makespan(plan, size, 0.05)
    time = row['time']
    results.loc[len(results)] = [size, planner, plan, makespan, reactive, time]
results.to_csv('../Data/heft/DynHeteroResources_StHeteroCampaignsHEFT_inaccur_5perc.csv', index=False)

test_case = pd.read_csv('../Data/heft/DynHeteroResources_StHeteroCampaignsHEFT_inaccur_p5perc.csv')
results = pd.DataFrame(columns=['size','planner','plan','makespan', 'reactive','time'])

for idx, row in test_case.iterrows():
    size = row['size']
    planner = row['planner']
    plan = eval(row['plan'])
    makespan, reactive = get_makespan(plan, size, 0.05, positive=True)
    time = row['time']
    results.loc[len(results)] = [size, planner, plan, makespan, reactive, time]
results.to_csv('../Data/heft/DynHeteroResources_StHeteroCampaignsHEFT_inaccur_p5perc.csv', index=False)

test_case = pd.read_csv('../Data/heft/DynHeteroResources_StHeteroCampaignsHEFT_inaccur_10perc.csv')
results = pd.DataFrame(columns=['size','planner','plan','makespan', 'reactive','time'])

for idx, row in test_case.iterrows():
    size = row['size']
    planner = row['planner']
    plan = eval(row['plan'])
    makespan, reactive = get_makespan(plan, size, 0.10)
    time = row['time']
    results.loc[len(results)] = [size, planner, plan, makespan, reactive, time]
results.to_csv('../Data/heft/DynHeteroResources_StHeteroCampaignsHEFT_inaccur_10perc.csv', index=False)

test_case = pd.read_csv('../Data/heft/DynHeteroResources_StHeteroCampaignsHEFT_inaccur_p10perc.csv')
results = pd.DataFrame(columns=['size','planner','plan','makespan', 'reactive','time'])

for idx, row in test_case.iterrows():
    size = row['size']
    planner = row['planner']
    plan = eval(row['plan'])
    makespan, reactive = get_makespan(plan, size, 0.10, positive=True)
    time = row['time']
    results.loc[len(results)] = [size, planner, plan, makespan, reactive, time]
results.to_csv('../Data/heft/DynHeteroResources_StHeteroCampaignsHEFT_inaccur_p10perc.csv', index=False)

test_case = pd.read_csv('../Data/heft/DynHeteroResources_StHeteroCampaignsHEFT_inaccur_20perc.csv')
results = pd.DataFrame(columns=['size','planner','plan','makespan', 'reactive','time'])

for idx, row in test_case.iterrows():
    size = row['size']
    planner = row['planner']
    plan = eval(row['plan'])
    makespan, reactive = get_makespan(plan, size, 0.20)
    time = row['time']
    results.loc[len(results)] = [size, planner, plan, makespan, reactive, time]
results.to_csv('../Data/heft/DynHeteroResources_StHeteroCampaignsHEFT_inaccur_20perc.csv', index=False)

test_case = pd.read_csv('../Data/heft/DynHeteroResources_StHeteroCampaignsHEFT_inaccur_p20perc.csv')
results = pd.DataFrame(columns=['size','planner','plan','makespan', 'reactive','time'])

for idx, row in test_case.iterrows():
    size = row['size']
    planner = row['planner']
    plan = eval(row['plan'])
    makespan, reactive = get_makespan(plan, size, 0.20, positive=True)
    time = row['time']
    results.loc[len(results)] = [size, planner, plan, makespan, reactive, time]
results.to_csv('../Data/heft/DynHeteroResources_StHeteroCampaignsHEFT_inaccur_p20perc.csv', index=False)

test_case = pd.read_csv('../Data/heft/DynHeteroResources_StHeteroCampaignsHEFT_inaccur_30perc.csv')
results = pd.DataFrame(columns=['size','planner','plan','makespan', 'reactive','time'])

for idx, row in test_case.iterrows():
    size = row['size']
    planner = row['planner']
    plan = eval(row['plan'])
    makespan, reactive = get_makespan(plan, size, 0.30)
    time = row['time']
    results.loc[len(results)] = [size, planner, plan, makespan, reactive, time]
results.to_csv('../Data/heft/DynHeteroResources_StHeteroCampaignsHEFT_inaccur_30perc.csv', index=False)

test_case = pd.read_csv('../Data/heft/DynHeteroResources_StHeteroCampaignsHEFT_inaccur_p30perc.csv')
results = pd.DataFrame(columns=['size','planner','plan','makespan', 'reactive','time'])

for idx, row in test_case.iterrows():
    size = row['size']
    planner = row['planner']
    plan = eval(row['plan'])
    makespan, reactive = get_makespan(plan, size, 0.30, positive=True)
    time = row['time']
    results.loc[len(results)] = [size, planner, plan, makespan, reactive, time]
results.to_csv('../Data/heft/DynHeteroResources_StHeteroCampaignsHEFT_inaccur_p30perc.csv', index=False)

test_case = pd.read_csv('../Data/heft/DynHeteroResources_StHeteroCampaignsHEFT_inaccur_40perc.csv')
results = pd.DataFrame(columns=['size','planner','plan','makespan', 'reactive','time'])

for idx, row in test_case.iterrows():
    size = row['size']
    planner = row['planner']
    plan = eval(row['plan'])
    makespan, reactive = get_makespan(plan, size, 0.40)
    time = row['time']
    results.loc[len(results)] = [size, planner, plan, makespan, reactive, time]
results.to_csv('../Data/heft/DynHeteroResources_StHeteroCampaignsHEFT_inaccur_40perc.csv', index=False)

test_case = pd.read_csv('../Data/heft/DynHeteroResources_StHeteroCampaignsHEFT_inaccur_p40perc.csv')
results = pd.DataFrame(columns=['size','planner','plan','makespan', 'reactive','time'])

for idx, row in test_case.iterrows():
    size = row['size']
    planner = row['planner']
    plan = eval(row['plan'])
    makespan, reactive = get_makespan(plan, size, 0.40, positive=True)
    time = row['time']
    results.loc[len(results)] = [size, planner, plan, makespan, reactive, time]
results.to_csv('../Data/heft/DynHeteroResources_StHeteroCampaignsHEFT_inaccur_p40perc.csv', index=False)

test_case = pd.read_csv('../Data/heft/DynHeteroResources_StHeteroCampaignsHEFT_inaccur_50perc.csv')
results = pd.DataFrame(columns=['size','planner','plan','makespan', 'reactive','time'])

for idx, row in test_case.iterrows():
    size = row['size']
    planner = row['planner']
    plan = eval(row['plan'])
    makespan, reactive = get_makespan(plan, size, 0.50)
    time = row['time']
    results.loc[len(results)] = [size, planner, plan, makespan, reactive, time]
results.to_csv('../Data/heft/DynHeteroResources_StHeteroCampaignsHEFT_inaccur_50perc.csv', index=False)

test_case = pd.read_csv('../Data/heft/DynHeteroResources_StHeteroCampaignsHEFT_inaccur_p50perc.csv')
results = pd.DataFrame(columns=['size','planner','plan','makespan', 'reactive','time'])

for idx, row in test_case.iterrows():
    size = row['size']
    planner = row['planner']
    plan = eval(row['plan'])
    makespan, reactive = get_makespan(plan, size, 0.50, positive=True)
    time = row['time']
    results.loc[len(results)] = [size, planner, plan, makespan, reactive, time]
results.to_csv('../Data/heft/DynHeteroResources_StHeteroCampaignsHEFT_inaccur_p50perc.csv', index=False)

test_case = pd.read_csv('../Data/heft/DynHeteroResources_StHeteroCampaignsHEFT_inaccur_60perc.csv')
results = pd.DataFrame(columns=['size','planner','plan','makespan', 'reactive','time'])

for idx, row in test_case.iterrows():
    size = row['size']
    planner = row['planner']
    plan = eval(row['plan'])
    makespan, reactive = get_makespan(plan, size, 0.60)
    time = row['time']
    results.loc[len(results)] = [size, planner, plan, makespan, reactive, time]
results.to_csv('../Data/heft/DynHeteroResources_StHeteroCampaignsHEFT_inaccur_60perc.csv', index=False)

test_case = pd.read_csv('../Data/heft/DynHeteroResources_StHeteroCampaignsHEFT_inaccur_p60perc.csv')
results = pd.DataFrame(columns=['size','planner','plan','makespan', 'reactive','time'])

for idx, row in test_case.iterrows():
    size = row['size']
    planner = row['planner']
    plan = eval(row['plan'])
    makespan, reactive = get_makespan(plan, size, 0.60, positive=True)
    time = row['time']
    results.loc[len(results)] = [size, planner, plan, makespan, reactive, time]
results.to_csv('../Data/heft/DynHeteroResources_StHeteroCampaignsHEFT_inaccur_p60perc.csv', index=False)

test_case = pd.read_csv('../Data/heft/DynHeteroResources_StHeteroCampaignsHEFT_inaccur_70perc.csv')
results = pd.DataFrame(columns=['size','planner','plan','makespan', 'reactive','time'])

for idx, row in test_case.iterrows():
    size = row['size']
    planner = row['planner']
    plan = eval(row['plan'])
    makespan, reactive = get_makespan(plan, size, 0.70)
    time = row['time']
    results.loc[len(results)] = [size, planner, plan, makespan, reactive, time]
results.to_csv('../Data/heft/DynHeteroResources_StHeteroCampaignsHEFT_inaccur_70perc.csv', index=False)

test_case = pd.read_csv('../Data/heft/DynHeteroResources_StHeteroCampaignsHEFT_inaccur_p70perc.csv')
results = pd.DataFrame(columns=['size','planner','plan','makespan', 'reactive','time'])

for idx, row in test_case.iterrows():
    size = row['size']
    planner = row['planner']
    plan = eval(row['plan'])
    makespan, reactive = get_makespan(plan, size, 0.70, positive=True)
    time = row['time']
    results.loc[len(results)] = [size, planner, plan, makespan, reactive, time]
results.to_csv('../Data/heft/DynHeteroResources_StHeteroCampaignsHEFT_inaccur_p70perc.csv', index=False)

test_case = pd.read_csv('../Data/heft/DynHeteroResources_StHeteroCampaignsHEFT_inaccur_80perc.csv')
results = pd.DataFrame(columns=['size','planner','plan','makespan', 'reactive','time'])

for idx, row in test_case.iterrows():
    size = row['size']
    planner = row['planner']
    plan = eval(row['plan'])
    makespan, reactive = get_makespan(plan, size, 0.80)
    time = row['time']
    results.loc[len(results)] = [size, planner, plan, makespan, reactive, time]
results.to_csv('../Data/heft/DynHeteroResources_StHeteroCampaignsHEFT_inaccur_80perc.csv', index=False)

test_case = pd.read_csv('../Data/heft/DynHeteroResources_StHeteroCampaignsHEFT_inaccur_p80perc.csv')
results = pd.DataFrame(columns=['size','planner','plan','makespan', 'reactive','time'])

for idx, row in test_case.iterrows():
    size = row['size']
    planner = row['planner']
    plan = eval(row['plan'])
    makespan, reactive = get_makespan(plan, size, 0.80, positive=True)
    time = row['time']
    results.loc[len(results)] = [size, planner, plan, makespan, reactive, time]
results.to_csv('../Data/heft/DynHeteroResources_StHeteroCampaignsHEFT_inaccur_p80perc.csv', index=False)

In [ ]:
test_case = pd.read_csv('../Data/heft/DynHeteroResources_StHeteroCampaignsHEFT_inaccur_90perc.csv')
results = pd.DataFrame(columns=['size','planner','plan','makespan', 'reactive','time'])

for idx, row in test_case.iterrows():
    size = row['size']
    planner = row['planner']
    plan = eval(row['plan'])
    makespan, reactive = get_makespan(plan, size, 0.90)
    time = row['time']
    results.loc[len(results)] = [size, planner, plan, makespan, reactive, time]
results.to_csv('../Data/heft/DynHeteroResources_StHeteroCampaignsHEFT_inaccur_90perc.csv', index=False)

test_case = pd.read_csv('../Data/heft/DynHeteroResources_StHeteroCampaignsHEFT_inaccur_p90perc.csv')
results = pd.DataFrame(columns=['size','planner','plan','makespan', 'reactive','time'])

for idx, row in test_case.iterrows():
    size = row['size']
    planner = row['planner']
    plan = eval(row['plan'])
    makespan, reactive = get_makespan(plan, size, 0.90, positive=True)
    time = row['time']
    results.loc[len(results)] = [size, planner, plan, makespan, reactive, time]
results.to_csv('../Data/heft/DynHeteroResources_StHeteroCampaignsHEFT_inaccur_p90perc.csv', index=False)

In [ ]:
def get_makespan(curr_plan, num_resources, workflow_inaccur, positive=False):
    '''
    Calculate makespan
    '''
    under = False
    reactive_resource_usage = [0] * num_resources
    resource_usage = [0] * num_resources
    tmp_idx = [0] * num_resources
    for placement in curr_plan:
        workflow = placement[0]
        resource = placement[1]
        resource_id = resource['id']
        expected_finish = placement[3]
        perf = gauss(resource['performance'], resource['performance'] * 0.0644)
        if positive:
            inaccur = uniform(0, workflow_inaccur)
        else:
            inaccur = uniform(-workflow_inaccur, workflow_inaccur)
        exec_time = (workflow['num_oper'] * (1 + inaccur)) / perf
        reactive_resource_usage[resource_id - 1] += exec_time
        if (resource_usage[resource_id - 1] + exec_time) == expected_finish:
            #under=True
            print(resource['performance'], perf, inaccur)
        resource_usage[resource_id - 1] = max(resource_usage[resource_id - 1] + exec_time, expected_finish)
        if (reactive_resource_usage[resource_id - 1]) == resource_usage[resource_id - 1]:
            #under=True
            print(resource_id,resource['performance'], perf, inaccur)
           
        tmp_idx[resource_id - 1] += 1
    if under:
        print('Resource Unused', num_resources)
    return resource_usage, reactive_resource_usage,tmp_idx

In [ ]:
test_case = pd.read_csv('../Data/heft/DynHeteroResources_StHeteroCampaignsHEFT_inaccur_90perc.csv')
test = test_case.loc[599]
size = test['size']
planner = test['planner']
plan = eval(test['plan'])
res, ares, work = get_makespan(plan, size, 0.90)

In [ ]:
res = res[::4] + res[1::4] + res[2::4] + res[3::4]
ares = ares[::4] + ares[1::4] + ares[2::4] + ares[3::4]
work = work[::4] + work[1::4] + work[2::4] + work[3::4]

In [ ]:
print(max(res),max(ares))

In [ ]:
from matplotlib import pyplot as plt
from matplotlib import cm

blues = cm.get_cmap(plt.get_cmap('Blues'))
greens = cm.get_cmap(plt.get_cmap('Greens'))
reds = cm.get_cmap(plt.get_cmap('Reds'))
oranges = cm.get_cmap(plt.get_cmap('Oranges'))
purples = cm.get_cmap(plt.get_cmap('Purples'))
greys = cm.get_cmap(plt.get_cmap('Greys'))
set1 = cm.get_cmap(plt.get_cmap('Set1'))

# Use coordinated colors. These are the "Tableau 20" colors as
# RGB. Each pair is strong/light. For a theory of color
tableau20 = [(31 , 119, 180), (174, 199, 232), # blue        [ 0,1 ]
             (255, 127, 14 ), (255, 187, 120), # orange      [ 2,3 ]
             (44 , 160, 44 ), (152, 223, 138), # green       [ 4,5 ]
             (214, 39 , 40 ), (255, 152, 150), # red         [ 6,7 ]
             (148, 103, 189), (197, 176, 213), # purple      [ 8,9 ]
             (140, 86 , 75 ), (196, 156, 148), # brown       [10,11]
             (227, 119, 194), (247, 182, 210), # pink        [12,13]
             (188, 189, 34 ), (219, 219, 141), # yellow      [14,15]
             (23 , 190, 207), (158, 218, 229), # cyan        [16,17]
             (65 , 68 , 81 ), (96 , 99 , 106), # gray        [18,19]
             (127, 127, 127), (143, 135, 130), # gray        [20,21]
             (165, 172, 175), (199, 199, 199), # gray        [22,23]
             (207, 207, 207)]                   # gray        [24]
# Scale the RGB values to the [0, 1] range, which is the format
# matplotlib accepts.
for i in range(len(tableau20)):
    r, g, b = tableau20[i]
    tableau20[i] = (round(r/255.,1), round(g/255.,1), round(b/255.,1))



fig,axis = plt.subplots(nrows=1,ncols=1,figsize=(15,7.5))
#_ = axis.plot(range(1, 129), random_util, color=tableau20[0], marker='o',label='Random',linewidth=0)
_ = axis.plot(range(1, 129), res,   color=tableau20[2], marker='s',label='HEFT',linewidth=0)
#_ = axis.plot(range(1, 129), ga_util,     color=tableau20[4], marker='^',label='GA',linewidth=0)
#_ = axis.plot(range(1, 129), ga25_util,   color=tableau20[6], marker='^',label='GA-25',linewidth=0)
_ = axis.plot(range(1, 129), ares,   color=tableau20[8], marker='^',label='GA-50',linewidth=0)
#_ = axis.plot(range(1, 129), l2ff_util,   color=tableau20[10], marker='s',label='L2FF',linewidth=0)
# _ = axis.fill_between(workflows_sizes, min_vals, max_vals, color=greens(250), alpha=0.2, label='RANDOM Max-Min')
# _ = axis.set_yscale('symlog')
_ = axis.set_ylabel('Resource Utilization',fontsize=20)
_ = axis.set_xlabel('Resource ID',fontsize=20)
_ = axis.set_xticklabels(axis.get_xticks().astype('int').tolist(),fontsize=18)
_ = axis.set_yticklabels(np.around(axis.get_yticks().tolist(), decimals=2),fontsize=18)
_ = axis.grid('on')
#_ = axis.set_title('Makespan comparison of static homogeneous campaigns\non static homogeneous resources', fontsize=22)
#_ = axis.text(6, 1.1, 'Bridges like', fontsize=20)
#_ = axis.text(40, 1.1, 'Comet like', fontsize=20)
#_ = axis.text(70, 1.1, 'Frontera like', fontsize=20)
#_ = axis.text(100, 1.1, 'Stampede 2 like', fontsize=20)
_ = axis.axvline(x=0.5, ymin=0, ymax=1.0, linestyle=':', color='black')
_ = axis.axvline(x=32.5, ymin=0, ymax=1.0, linestyle=':', color='black')
_ = axis.axvline(x=64.5, ymin=0, ymax=1.0, linestyle=':', color='black')
_ = axis.axvline(x=96.5, ymin=0, ymax=1.0, linestyle=':', color='black')
_ = axis.axvline(x=128.5, ymin=0, ymax=1.0, linestyle=':', color='black')
_ = axis.legend(fontsize=20)
#_ = fig.savefig('../Figures/HeteroResources_StHeteroCampaigns_Utils.pdf',bbox_inches='tight')
#fig.savefig('StHomoCampaigns_4StHomoResources.pdf',bbox_inches='tight')

In [ ]:
fig,axis = plt.subplots(nrows=1,ncols=1,figsize=(15,7.5))
#_ = axis.plot(range(1, 129), random_util, color=tableau20[0], marker='o',label='Random',linewidth=0)
_ = axis.plot(range(1, 129), work,   color=tableau20[2], marker='s',label='HEFT',linewidth=0)
#_ = axis.plot(range(1, 129), ga_util,     color=tableau20[4], marker='^',label='GA',linewidth=0)
#_ = axis.plot(range(1, 129), ga25_util,   color=tableau20[6], marker='^',label='GA-25',linewidth=0)
#_ = axis.plot(range(1, 129), ares,   color=tableau20[8], marker='^',label='GA-50',linewidth=0)
#_ = axis.plot(range(1, 129), l2ff_util,   color=tableau20[10], marker='s',label='L2FF',linewidth=0)
# _ = axis.fill_between(workflows_sizes, min_vals, max_vals, color=greens(250), alpha=0.2, label='RANDOM Max-Min')
# _ = axis.set_yscale('symlog')
_ = axis.set_ylabel('Resource Utilization',fontsize=20)
_ = axis.set_xlabel('Resource ID',fontsize=20)
_ = axis.set_xticklabels(axis.get_xticks().astype('int').tolist(),fontsize=18)
_ = axis.set_yticklabels(np.around(axis.get_yticks().tolist(), decimals=2),fontsize=18)
_ = axis.grid('on')
#_ = axis.set_title('Makespan comparison of static homogeneous campaigns\non static homogeneous resources', fontsize=22)
_ = axis.text(6, 1.1, 'Bridges like', fontsize=20)
_ = axis.text(40, 1.1, 'Comet like', fontsize=20)
_ = axis.text(70, 1.1, 'Frontera like', fontsize=20)
_ = axis.text(100, 1.1, 'Stampede 2 like', fontsize=20)
_ = axis.axvline(x=0.5, ymin=0, ymax=1.0, linestyle=':', color='black')
_ = axis.axvline(x=32.5, ymin=0, ymax=1.0, linestyle=':', color='black')
_ = axis.axvline(x=64.5, ymin=0, ymax=1.0, linestyle=':', color='black')
_ = axis.axvline(x=96.5, ymin=0, ymax=1.0, linestyle=':', color='black')
_ = axis.axvline(x=128.5, ymin=0, ymax=1.0, linestyle=':', color='black')
_ = axis.legend(fontsize=20)
#_ = fig.savefig('../Figures/HeteroResources_StHeteroCampaigns_Utils.pdf',bbox_inches='tight')
#fig.savefig('StHomoCampaigns_4StHomoResources.pdf',bbox_inches='tight')

In [ ]:
ares[17]